In [94]:
camera_csv = pd.read_csv('/export/jupyterlab/data/udacity-challenge-2/Ch2_002_export_png/interpolated.csv')

camera_csv

,index,timestamp,width,height,frame_id,filename,angle,torque,speed,lat,long,alt
0,2016-11-17 23:10:15.873184606,1479424215873184606,640,480,left_camera,left/1479424215873184606.png,0.000360,0.375000,23.003350,37.545269,-122.326485,8.116664
1,2016-11-17 23:10:15.877284689,1479424215877284689,640,480,right_camera,right/1479424215877284689.png,0.000717,0.375000,23.003919,37.545269,-122.326485,8.123680
2,2016-11-17 23:10:15.880976321,1479424215880976321,640,480,center_camera,center/1479424215880976321.png,0.001039,0.375000,23.004431,37.545269,-122.326492,8.128418
3,2016-11-17 23:10:15.922817911,1479424215922817911,640,480,left_camera,left/1479424215922817911.png,0.003491,0.394737,23.007310,37.545261,-122.326500,8.139066
4,2016-11-17 23:10:15.927281227,1479424215927281227,640,480,right_camera,right/1479424215927281227.png,0.003491,0.380823,23.006074,37.545261,-122.326500,8.144778
...,...,...,...,...,...,...,...,...,...,...,...,...
101391,2016-11-17 23:49:32.293518518,1479426572293518518,640,480,center_camera,center/1479426572293518518.png,0.000000,0.024914,11.738325,37.548805,-122.315819,-19.648416
101392,2016-11-17 23:49:32.303971946,1479426572303971946,640,480,right_camera,right/1479426572303971946.png,0.000000,0.057556,11.741227,37.548805,-122.315819,-19.648843
101393,2016-11-17 23:49:32.317632186,1479426572317632186,640,480,left_camera,left/1479426572317632186.png,0.000000,0.062500,11.743342,37.548805,-122.315811,-19.650489
101394,2016-11-17 23:49:32.343447996,1479426572343447996,640,480,center_camera,center/1479426572343447996.png,0.000000,0.011082,11.744445,37.548809,-122.315811,-19.654094


In [253]:
# data loading

from __future__ import print_function, division
import pandas as pd
import matplotlib.pyplot as plt
from skimage import io, transform
import os
import numpy as np

# defining our own Dataset class

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

class UdacityDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None, select_camera=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            select_camera (string): 'left_ / right_ / center_camera'
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        camera_csv = pd.read_csv(csv_file)
        if select_camera:
            camera_csv = camera_csv[camera_csv['frame_id']==select_camera]
        self.camera_csv = camera_csv
        self.root_dir = root_dir
        self.transform = transform
    
    def __len__(self):
        return len(self.camera_csv)
    
    def read_data(self, idx):
        path = os.path.join(self.root_dir, self.camera_csv['filename'].iloc[idx])
        image = io.imread(path)
        timestamp = self.camera_csv['timestamp'].iloc[idx]
        frame_id = self.camera_csv['frame_id'].iloc[idx]
        angle = self.camera_csv['angle'].iloc[idx]
            
        return image, timestamp, frame_id, angle
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        if idx is int:
            image, timestamp, frame_id, angle = read_data(idx)
        else:
            image = []
            timestamp = []
            frame_id = []
            angle = []
            
            for i in idx:
                im, ti, fr, an = self.read_data(i)
                image.append(im)
                timestamp.append(ti)
                frame_id.append(fr)
                angle.append(an)
            
        if self.transform:
            image = self.transform(np.array(image))
            
        sample = {'image': image,
                  'timestamp': timestamp,
                  'frame_id': frame_id,
                  'angle': angle}
            
        return sample

In [254]:
# test the UdacityDataset class

def show_sample(sample):
    plt.imshow(sample['image'])
    plt.title("cam: {} angle: {}".format(sample['frame_id'], sample['angle']))

# to load as normal images:    

# udacity_dataset = UdacityDataset(csv_file='/export/jupyterlab/data/udacity-challenge-2/Ch2_002_export/interpolated.csv',
#                                  root_dir='/export/jupyterlab/data/udacity-challenge-2/Ch2_002_export/',
#                                  transform=None,
#                                  select_camera='center_camera')
# print(udacity_dataset.camera_csv['filename'].iloc[0])

# fig = plt.figure()

# for i in range(3):
#     sample = udacity_dataset[i]
#     ax = plt.subplot(1, 4, i%4+1)
#     plt.tight_layout()
#     ax.axis('off')
#     show_sample(sample)
#     plt.pause(0.001)
    
#     if i % 4 == 0:
#         plt.show()

# to load as tensors

udacity_dataset = UdacityDataset(csv_file='/export/jupyterlab/data/udacity-challenge-2/Ch2_002_export/interpolated.csv',
                                 root_dir='/export/jupyterlab/data/udacity-challenge-2/Ch2_002_export/',
                                 transform=transforms.Compose([transforms.ToTensor()]))

# print("type of image data:")
# print(type(udacity_dataset[0]['image']))
# print(udacity_dataset[1]['image'].shape)

# using DataLoader and batch_size
# this allows multithreading so it has a higher efficiency

sampler = SequentialSampler(udacity_dataset)
dataloader = DataLoader(udacity_dataset, sampler=BatchSampler(sampler, batch_size=1, drop_last=False))
# dataloader = DataLoader(udacity_dataset, batch_size=4)
for i_batch, sample_batched in enumerate(dataloader):
    print(type(sample_batched['image']))
    print(sample_batched['image'].shape)
    break

ValueError: pic should be 2/3 dimensional. Got 4 dimensions.

In [185]:
# generate Batch with consecutive frames taken from input data

from torch.utils.data import Sampler
import random

class ConsecutiveBatchSampler(Sampler):
    
    def __init__(self, data_source, batch_size, padding, last_batch=None, shuffle=True):
        r""" Sampler to generate consecutive Batches
        
        Args:
            data_source: Source of data
            batch_size: Number of frames in each batch
            padding: Number of frames to insert before each batch
            last_batch: Operation to the last batch. Use 'drop', 'fill' or None
            shuffle: Wether to shuffle the data
        """
        self.data_source = data_source
        
        assert batch_size >= 1, "Invalid batch size: {}".format(batch_size)
        self.batch_size = batch_size
        
        assert padding >= 0, "Invalid padding size: {}".format(padding)
        self.padding = padding
        
        assert last_batch in ['drop', 'fill', None], "Invalid option last_batch={}".format(last_batch)
        self.last_batch=last_batch
        
        self.shuffle = shuffle
    
    def __iter__(self):
        length = len(self.data_source)
        start_indices = list(range(0, length, self.batch_size))
        if self.shuffle:
            random.shuffle(start_indices)
        
        for ind in start_indices:
            batch = []
            
            if ind - self.padding < 0:
                batch.extend([0]*(self.padding - ind) + list(range(0, ind)))
            else:
                batch.extend(list(range(ind - self.padding, ind)))
            
            if ind + self.batch_size > length:
                if self.last_batch == 'drop':
                    continue
                elif self.last_batch == 'fill':
                    batch.extend(list(range(ind, length)) + list(range(0, self.batch_size-(length-ind))))
                else:
                    batch.extend(list(range(ind, length)))
            else:
                batch.extend(list(range(ind, ind+self.batch_size)))
            
            yield batch
    
    def __len__(self):
        length = len(self.data_source)
        
        if length % batch_size == 0 or self.last_batch == 'fill':
            return length // batch_size
        
        if self.last_batch == 'drop':
            return length // batch_size
        
        return length // batch_size + 1
            

In [197]:
# # test the ConsecutiveBatchSampler

from torch.utils.data import BatchSampler, SequentialSampler, RandomSampler

dummy_dataset = list(range(0, 20))
cbs = ConsecutiveBatchSampler(data_source=dummy_dataset, batch_size=4, shuffle=True, last_batch=None, padding = 0)
print(list([a for a in cbs]))
bs = BatchSampler(RandomSampler(dummy_dataset), batch_size=4, drop_last=False)
print(list([a for a in bs]))
    
#test the sampler on DataLoader

# batch_sampler = ConsecutiveBatchSampler(data_source=udacity_dataset, batch_size=4, shuffle=True, last_batch=None, padding=1)
# dataloader = DataLoader(udacity_dataset, sampler=batch_sampler, num_workers=1)
# for i_batch, sample_batched in enumerate(dataloader):
#     print(sample_batched['image'].shape)
#     break

[[12, 13, 14, 15], [4, 5, 6, 7], [0, 1, 2, 3], [16, 17, 18, 19], [8, 9, 10, 11]]
[[4, 16, 9, 3], [2, 7, 8, 13], [5, 10, 11, 14], [15, 6, 18, 19], [1, 0, 17, 12]]


In [ ]:
# Model 1
import torch.nn as nn

class Conv3DModel(nn.Module, W_in=640, H_in=480):
    def __init__(self):
        super(Conv3DModel, self).__init__()
        
        
    
    def forward(self, x):
        pass
    
